In [1]:
import pickle
import pandas as pd 
import gc
import os
import pandas as pd
import numpy as np
import lightgbm
from sklearn.metrics import precision_score,recall_score,roc_auc_score,classification_report

/home/liwenliang/anaconda3/envs/rob/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Multi-Classification
user_embedding*item_embedding->score(x)
->task(y)
->lightGBM

## train

In [2]:
data_i = pd.read_csv('./datasets/data.csv')
events = pd.read_csv('./datasets/event.csv')
user_id = events['user_id']
data_i = data_i.set_index(user_id)
data_i = data_i.drop(events[events['labels']=='1,1,0,0,1,0,0,0,1'].index,axis = 0)
data_i = data_i.drop(events[events['labels']=='0,1,0,1,0,0,1,0,0'].index,axis = 0)
# data_i = data_i.drop(bb[bb.iloc[:,6]!=54240].index,axis = 0)
y = events['labels'].drop(events[events['labels']=='1,1,0,0,1,0,0,0,1'].index,axis = 0)
y = y.drop(events[events['labels']=='0,1,0,1,0,0,1,0,0'].index,axis = 0)
# y = y.drop(bb[bb.iloc[:,6]!=54240].index,axis = 0)
data_i

,scores,scores.1,scores.2,scores.3,scores.4,scores.5,scores.6,scores.7,scores.8,group_scores,...,interval,price,price.1,price.2,price.3,price.4,price.5,price.6,price.7,price.8
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.334250,0.312646,0.310058,0.317037,0.277637,0.314117,0.325306,0.340136,0.371542,0.318985,...,0.80,2954.0,1978.0,295.0,559.0,1793.0,1429.0,2199.0,690.0,3777.0
2,0.405298,0.322350,0.436186,0.375337,0.176343,0.489126,0.577481,0.312774,0.486421,0.387945,...,0.75,2954.0,1486.0,1271.0,1787.0,1793.0,1430.0,11997.0,497.0,7000.0
3,0.356495,0.343906,0.362385,0.349160,0.258404,0.350972,0.232758,0.494536,0.368001,0.354262,...,0.65,637.0,1486.0,295.0,5208.0,1780.0,4578.0,1768.0,1756.0,5558.0
4,0.332843,0.328229,0.307702,0.279055,0.376070,0.364445,0.334881,0.465579,0.397959,0.322925,...,0.75,593.0,478.0,2954.0,239.0,1488.0,1432.0,1387.0,6037.0,7000.0
5,0.417497,0.405888,0.356196,0.338698,0.281830,0.410781,0.357581,0.546216,0.378840,0.393194,...,0.70,1479.0,2954.0,478.0,279.0,2770.0,1787.0,1959.0,7000.0,1065.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260083,0.373168,0.363249,0.368822,0.308188,0.287073,0.259102,0.445450,0.303088,0.347769,0.368413,...,0.85,1207.0,618.0,946.0,1117.0,1785.0,676.0,10594.0,4079.0,2383.0
260084,0.436682,0.295297,0.309889,0.349857,0.352108,0.311665,0.321082,0.311732,0.454284,0.347289,...,0.65,1222.0,475.0,199.0,2492.0,558.0,1793.0,943.0,1788.0,7000.0
260085,0.347315,0.435804,0.332577,0.399524,0.317818,0.313250,0.409925,0.455442,0.403814,0.371899,...,0.65,295.0,693.0,593.0,897.0,349.0,2769.0,6037.0,3500.0,3777.0


In [3]:
data_i_v = pd.read_csv('./datasets/data_v.csv')
x_valid = data_i_v
x_valid

,scores,scores.1,scores.2,scores.3,scores.4,scores.5,scores.6,scores.7,scores.8,group_scores,...,interval,price,price.1,price.2,price.3,price.4,price.5,price.6,price.7,price.8
0,0.361060,0.319881,0.295002,0.303006,0.344006,0.323102,0.399915,0.353739,0.412664,0.325315,...,0.85,757.0,593.0,1207.0,898.0,359.0,358.0,3777.0,1188.0,7000.0
1,0.365978,0.430513,0.418576,0.423834,0.290246,0.408647,0.329504,0.369062,0.480752,0.405022,...,0.85,637.0,1207.0,946.0,1557.0,2770.0,2181.0,1181.0,2199.0,3777.0
2,0.428479,0.464028,0.278901,0.409794,0.336164,0.323324,0.342695,0.334072,0.362839,0.390469,...,0.75,1207.0,693.0,475.0,1400.0,1440.0,2492.0,4079.0,2513.0,497.0
3,0.317010,0.337044,0.336006,0.324239,0.379250,0.316856,0.333559,0.391231,0.284750,0.330020,...,0.70,1207.0,636.0,693.0,559.0,3873.0,2148.0,2199.0,3500.0,2235.0
4,0.289547,0.344884,0.314824,0.337340,0.325776,0.236394,0.336476,0.299571,0.375464,0.316419,...,0.70,1486.0,199.0,295.0,646.0,1400.0,1793.0,549.0,1387.0,6041.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206249,-0.001540,-0.010733,-0.007529,-0.002096,-0.001996,-0.000172,0.000051,-0.006069,0.002672,-0.006601,...,0.60,295.0,589.0,946.0,836.0,448.0,1117.0,3777.0,745.0,3500.0
206250,0.408904,0.365829,0.362110,0.358949,0.377627,0.324649,0.510981,0.410309,0.304880,0.378948,...,0.65,593.0,596.0,589.0,803.0,1195.0,5208.0,11768.0,2000.0,2389.0
206251,-0.006049,0.006241,0.003537,-0.001775,0.003555,0.000552,0.002672,0.000217,0.002841,0.001243,...,0.60,1479.0,478.0,1486.0,3338.0,559.0,1440.0,3500.0,4079.0,497.0
206252,0.282106,0.369575,0.603263,0.207410,0.362853,0.267989,0.252864,0.398873,0.569444,0.418315,...,0.65,1962.0,876.0,2954.0,279.0,896.0,359.0,8524.0,3777.0,5396.0


In [ ]:
# from sklearn import metrics
# kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
# # kf = KFold(n_splits = 5, shuffle = True, random_state = 0)
# # y_pred = np.zeros(len(x_test))

# for fold,(train_index,val_index) in enumerate(kf.split(X_train,Y_train)):
#     x_train,x_val = X_train.iloc[train_index],X_train.iloc[val_index]
# #     Y_train = Y_train.str.split(',')
#     y_train,y_val = Y_train.iloc[train_index],Y_train.iloc[val_index]
#     train_set = lgb.Dataset(x_train,y_train)
#     val_set = lgb.Dataset(x_val,y_val)

## model

In [4]:
model = lightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=256,reg_alpha=0.0,reg_lambda=1,
max_depth=8, n_estimators=80,objective='multiclassova' ,num_class = 22,
learning_rate=0.08,random_state=20,n_jobs=4)#,class_weight = weights)#,lambda_l1 = 0.3,lambda_l2 = 0.4)

In [5]:
n_user = len(data_i)
left = 0.9
right = 1

x_train = pd.concat([data_i[:int(n_user*left)],data_i[int(n_user*right):]])
x_test = data_i[int(n_user*left):int(n_user*right)]
# x_test = data_i[int(n_user*left):int(n_user*right)]
y_train = pd.concat([y[:int(n_user*left)],y[int(n_user*right):]])
y_test  = y[int(n_user*left):int(n_user*right)]
# y_test  = y[int(n_user*left):int(n_user*right)]
model.fit(x_train, y_train, eval_set =(x_test,y_test) ,eval_metric = 'multi_error')#,categorical_feature=categorical)#['item_class','user'])
    # A={}
# def unqiue_element(x):
#     a = len(np.unique(x))
#     A[x.name] = a
# df[NUMERIC_COLS].apply(unqiue_element)
# print(sorted ( A.items ( ) , key=lambda x: x[1], reverse=True))
predictions = model.predict(x_test)
print( '正确率: ',model.score(x_test, y_test))
print( '精确率: ',precision_score(y_test,predictions,average = 'weighted'))
labels = ['0,0,0,0,0,0,0,0,0','0,0,1,0,0,0,0,0,0','0,1,0,0,0,0,0,0,0','0,1,1,0,0,0,0,0,0','1,0,0,0,0,0,0,0,0','1,0,1,0,0,0,0,0,0','1,1,0,0,0,0,0,0,0','1,1,1,0,0,0,0,0,0','1,1,1,0,0,1,0,0,0','1,1,1,0,1,0,0,0,0','1,1,1,0,1,1,0,0,0','1,1,1,1,0,0,0,0,0','1,1,1,1,0,1,0,0,0','1,1,1,1,1,0,0,0,0','1,1,1,1,1,1,0,0,0','1,1,1,1,1,1,0,0,1','1,1,1,1,1,1,0,1,0','1,1,1,1,1,1,0,1,1','1,1,1,1,1,1,1,0,0','1,1,1,1,1,1,1,0,1','1,1,1,1,1,1,1,1,0','1,1,1,1,1,1,1,1,1']
# labels = [1,2,3,4,5,6,7]
print('评估:',classification_report(y_test,predictions,target_names=labels))



# from sklearn.model_selection import StratifiedKFold,KFold
# from sklearn.metrics import precision_score,recall_score,roc_auc_score,classification_report
# # kf = StratifiedKFold(n_splits = 5)#, shuffle = True, random_state = 40)
# kf = KFold(n_splits = 10, shuffle = True, random_state = 0)
# # y_pred = np.zeros(len(x_test))

# for fold,(train_index,val_index) in enumerate(kf.split(data_i,y)):
# #     print(train_index,'\n',val_index)
#     x_train,x_test = data_i.iloc[train_index],data_i.iloc[val_index]
# #     Y_train = Y_train.str.split(',')
#     y_train,y_test = y.iloc[train_index],y.iloc[val_index]
#     model.fit(x_train, y_train,categorical_feature=categorical_feature)# ,categorical_feature=['item_class','user'])
#         # A={}
#     # def unqiue_element(x):
#     #     a = len(np.unique(x))
#     #     A[x.name] = a
#     # df[NUMERIC_COLS].apply(unqiue_element)
#     # print(sorted ( A.items ( ) , key=lambda x: x[1], reverse=True))
#     predictions = model.predict(x_test)
#     print( '正确率: ',model.score(x_test, y_test))
#     print( '精确率: ',precision_score(y_test,predictions,average = 'weighted'))
#     labels = ['0,0,0,0,0,0,0,0,0','0,0,1,0,0,0,0,0,0','0,1,0,0,0,0,0,0,0','0,1,1,0,0,0,0,0,0','1,0,0,0,0,0,0,0,0','1,0,1,0,0,0,0,0,0','1,1,0,0,0,0,0,0,0','1,1,1,0,0,0,0,0,0','1,1,1,0,0,1,0,0,0','1,1,1,0,1,0,0,0,0','1,1,1,0,1,1,0,0,0','1,1,1,1,0,0,0,0,0','1,1,1,1,0,1,0,0,0','1,1,1,1,1,0,0,0,0','1,1,1,1,1,1,0,0,0','1,1,1,1,1,1,0,0,1','1,1,1,1,1,1,0,1,0','1,1,1,1,1,1,0,1,1','1,1,1,1,1,1,1,0,0','1,1,1,1,1,1,1,0,1','1,1,1,1,1,1,1,1,0','1,1,1,1,1,1,1,1,1']
#     # labels = [1,2,3,4,5,6,7]
#     print('评估:',classification_report(y_test,predictions,target_names=labels))

[1]	valid_0's multi_error: 0.764005	valid_0's multi_logloss: 2.50027
[2]	valid_0's multi_error: 0.701872	valid_0's multi_logloss: 2.40255
[3]	valid_0's multi_error: 0.649429	valid_0's multi_logloss: 2.32921
[4]	valid_0's multi_error: 0.6219	valid_0's multi_logloss: 2.27037
[5]	valid_0's multi_error: 0.604483	valid_0's multi_logloss: 2.22172
[6]	valid_0's multi_error: 0.59637	valid_0's multi_logloss: 2.18152
[7]	valid_0's multi_error: 0.590603	valid_0's multi_logloss: 2.14629
[8]	valid_0's multi_error: 0.586297	valid_0's multi_logloss: 2.1157
[9]	valid_0's multi_error: 0.581952	valid_0's multi_logloss: 2.08954
[10]	valid_0's multi_error: 0.578338	valid_0's multi_logloss: 2.06634
[11]	valid_0's multi_error: 0.575801	valid_0's multi_logloss: 2.04587
[12]	valid_0's multi_error: 0.574263	valid_0's multi_logloss: 2.02825
[13]	valid_0's multi_error: 0.573263	valid_0's multi_logloss: 2.01263
[14]	valid_0's multi_error: 0.571802	valid_0's multi_logloss: 1.99864
[15]	valid_0's multi_error: 0.570

In [46]:
# categorical = []
# for i in range(18):
#     if i == 0:
#         categorical.append('item_class')
#         continue
#     categorical.append('item_class.'+str(i))

# for i in [0,2,3,6,7,8]:
#     if i == 0:
#         categorical.append('user')
#         continue
#     categorical.append('user.'+str(i))

# weights = {'0,0,0,0,0,0,0,0,0':1,'0,0,1,0,0,0,0,0,0':2,'0,1,0,0,0,0,0,0,0':1.8,'0,1,1,0,0,0,0,0,0':3.6,
#            '1,0,0,0,0,0,0,0,0':1.5,'1,0,1,0,0,0,0,0,0':3,'1,1,0,0,0,0,0,0,0':2.2,'1,1,1,0,0,0,0,0,0':2.9,
#            '1,1,1,0,0,1,0,0,0':1.9,'1,1,1,0,1,0,0,0,0':2,'1,1,1,0,1,1,0,0,0':2,'1,1,1,1,0,0,0,0,0':1.8,
#            '1,1,1,1,0,1,0,0,0':2,'1,1,1,1,1,0,0,0,0':1.6,'1,1,1,1,1,1,0,0,0':3,'1,1,1,1,1,1,0,0,1':1.5,
#            '1,1,1,1,1,1,0,1,0':1.2,'1,1,1,1,1,1,0,1,1':1.05,'1,1,1,1,1,1,1,0,0':1.05,'1,1,1,1,1,1,1,0,1':1,
#            '1,1,1,1,1,1,1,1,0':0.9,'1,1,1,1,1,1,1,1,1':0.9}

# weights = {'0,0,0,0,0,0,0,0,0':0.8,'0,0,1,0,0,0,0,0,0':1,'0,1,0,0,0,0,0,0,0':1,'0,1,1,0,0,0,0,0,0':1.2,
#            '1,0,0,0,0,0,0,0,0':1,'1,0,1,0,0,0,0,0,0':1.2,'1,1,0,0,0,0,0,0,0':1,'1,1,1,0,0,0,0,0,0':1.2,
#            '1,1,1,0,0,1,0,0,0':1,'1,1,1,0,1,0,0,0,0':1,'1,1,1,0,1,1,0,0,0':1,'1,1,1,1,0,0,0,0,0':1,
#            '1,1,1,1,0,1,0,0,0':1,'1,1,1,1,1,0,0,0,0':1,'1,1,1,1,1,1,0,0,0':1.2,'1,1,1,1,1,1,0,0,1':1,
#            '1,1,1,1,1,1,0,1,0':1,'1,1,1,1,1,1,0,1,1':1,'1,1,1,1,1,1,1,0,0':1.,'1,1,1,1,1,1,1,0,1':1,
#            '1,1,1,1,1,1,1,1,0':0.8,'1,1,1,1,1,1,1,1,1':0.8}

In [47]:
# import lightgbm
# import xgboost
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from mlxtend.classifier import StackingCVClassifier
# from sklearn import metrics
# # 基分类器3个：DecisionTreeClassifier， GradientBoostingClassifier，RandomForestClassifier
# # 目标分类器：LogisticRegression
# # 使用3折交叉验证
# basemodel1 = lightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=256,reg_alpha=0.0,reg_lambda=1,
#                                      max_depth=8, n_estimators=80,objective='multiclassova' ,num_class = 22,
#                                      learning_rate=0.08,random_state=20,n_jobs=4)
# basemodel2 = xgboost.XGBClassifier(reg_alpha=0.0,reg_lambda=1,max_depth=6, n_estimators=80,
#                                    objective='multi:softmax' ,num_class = 22,learning_rate=0.08,random_state=20)
# basemodel3 = RandomForestClassifier(max_features=None,random_state=20, n_estimators=80,criterion='gini' ,max_depth=10,verbose=0,
#                                     warm_start=False,class_weight=None)

# lr = LogisticRegression()
# sclf = StackingCVClassifier(classifiers=[basemodel1, basemodel2, basemodel3], meta_classifier=lr,
#                                    use_probas=True, verbose=2, cv=5)

# # basemodel1 = tree.DecisionTreeClassifier()
# # basemodel2 = GradientBoostingClassifier()
# # basemodel3 = RandomForestClassifier()
# # lr = LogisticRegression()
# # sclf = StackingCVClassifier(classifiers=[basemodel1, basemodel2, basemodel3], meta_classifier=lr,
# #                                    use_probas=True, verbose=2, cv=3)
# sclf.fit(x_train, y_train)
# sclf_predict = sclf.predict(x_test)
# sclf_predict_proba = sclf.predict_proba(x_test)[:,1]
# print('评估:',classification_report(y_test,sclf_predict,target_names=labels))
# print('score',sclf.score(y_test, sclf_predict, sclf_predict_proba))

In [6]:
pd.DataFrame({'column':data_i.columns.tolist(),'importance':model.feature_importances_}).sort_values(by='importance' , ascending=False)[:50]

,column,importance
57,user.1,8922
65,user.9,8730
8,scores.8,8379
7,scores.7,7858
6,scores.6,7857
51,probability.7,7748
43,probability.5,7746
47,probability.6,7647
42,rare.5,7283
55,probability.8,7116


In [7]:
y_valid = model.predict(x_valid)
# y_valid = sclf.predict(x_valid)
aa = y_valid.tolist()
for n,i in enumerate(aa):
    i = i.split(',')
    aa[n] = ' '.join(i)
n_user = data_i_v.shape[0]
i = pd.DataFrame([i for i in range(1,n_user+1)],columns = ['id'])
aa = pd.DataFrame(aa,columns = ['category'])
res = pd.concat([i,aa],axis = 1)
res

,id,category
0,1,1 1 1 1 1 1 1 1 1
1,2,1 1 1 1 1 1 1 1 1
2,3,1 1 1 0 0 1 0 0 0
3,4,1 1 1 1 1 1 1 1 0
4,5,1 1 1 1 1 1 0 1 1
...,...,...
206249,206250,1 1 1 1 1 1 1 1 1
206250,206251,0 0 0 0 0 0 0 0 0
206251,206252,1 1 1 1 1 1 1 0 1
206252,206253,0 0 0 0 0 0 0 0 0


In [50]:
res.to_csv('./datasets/res.csv',sep = ',',index = False)